In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
from dateutil.parser import parse

import stock

%matplotlib inline

In [132]:
# 決算プロの決算短信csv(エクセルファイルより書き出し)を読み込む
kessan_table = pd.read_csv('/Users/Really/Stockyard/_dl_data/kabupro_20171112f.csv') # http://ke.kabupro.jp/doc/down40.htm

In [139]:
kessan_table['期首'] = kessan_table['期首'].apply(lambda x: parse(x).date())
kessan_table['期末'] = kessan_table['期末'].apply(lambda x: parse(x).date())
kessan_table['情報公開又は更新日'] = kessan_table['情報公開又は更新日'].apply(lambda x: parse(x).date())

In [140]:
# まとめてできないの？
# kessan_table[['期首', '期末', '情報公開又は更新日']] = pd.to_datetime(kessan_table[['期首', '期末', '情報公開又は更新日']], format='%Y-%m-%d')
kessan_table['期首'] = pd.to_datetime(kessan_table['期首'], format='%Y-%m-%d')
kessan_table['期末'] = pd.to_datetime(kessan_table['期末'], format='%Y-%m-%d')
kessan_table['情報公開又は更新日'] = pd.to_datetime(kessan_table['情報公開又は更新日'], format='%Y-%m-%d')

In [104]:
num_list = [
    '売上高', 
    '営業利益', 
    '経常利益', 
    '純利益', 
    '一株当り純利益',
    '希薄化後一株当り純利益', 
    '純資産又は株主資本', 
    '総資産', 
    '一株当り純資産', 
    '営業キャッシュフロー',
    '投資キャッシュフロー', 
    '財務キャッシュフロー'
]

In [133]:
kessan_table = kessan_table.astype(dict(zip(num_list, [str for i in range(len(num_list))])))
kessan_table[num_list] = kessan_table[num_list].applymap(lambda x: x.replace(',', ''))
kessan_table = kessan_table.astype(dict(zip(num_list, [float for i in range(len(num_list))])))

In [141]:
kessan_table

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
0,1301,極洋,日本基準,連結,2018年3月期,第2四半期,2017-04-01,2017-09-30,売上高,1.204580e+11,...,1.633000e+09,155.50,144.15,2.669200e+10,1.198060e+11,NaN,NaN,NaN,NaN,2017-11-06
1,1301,極洋,日本基準,連結,2018年3月期,第1四半期,2017-04-01,2017-06-30,売上高,5.684400e+10,...,7.540000e+08,71.80,66.56,2.556000e+10,1.074220e+11,NaN,NaN,NaN,NaN,2017-08-04
2,1301,極洋,日本基準,連結,2017年3月期,通期,2016-04-01,2017-03-31,売上高,2.365610e+11,...,2.422000e+09,230.66,213.01,2.539100e+10,9.739100e+10,2378.09,6.010000e+08,-1.998000e+09,1.050000e+08,2017-05-11
3,1301,極洋,日本基準,連結,2017年3月期,第3四半期,2016-04-01,2016-12-31,売上高,1.799750e+11,...,2.449000e+09,233.26,215.13,2.577900e+10,1.171680e+11,NaN,NaN,NaN,NaN,2017-02-10
4,1301,極洋,日本基準,連結,2017年3月期,第2四半期,2016-04-01,2016-09-30,売上高,1.095700e+11,...,1.186000e+09,112.99,104.16,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-06
5,1301,極洋,日本基準,連結,2017年3月期,第1四半期,2016-04-01,2016-06-30,売上高,5.220600e+10,...,5.510000e+08,52.47,48.37,NaN,NaN,NaN,NaN,NaN,NaN,2017-08-04
6,1301,極洋,日本基準,連結,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.266260e+11,...,1.799000e+09,171.33,157.94,2.306500e+10,9.460800e+10,2149.67,2.689000e+09,-5.114000e+09,2.482000e+09,2017-05-11
7,1301,極洋,日本基準,連結,2016年3月期,第3四半期,2015-04-01,2015-12-31,売上高,1.788900e+11,...,1.133000e+09,107.94,99.51,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-10
8,1301,極洋,日本基準,連結,2016年3月期,第2四半期,2015-04-01,2015-09-30,売上高,1.084990e+11,...,9.110000e+08,86.74,79.97,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-04
9,1301,極洋,日本基準,連結,2016年3月期,第1四半期,2015-04-01,2015-06-30,売上高,5.016800e+10,...,3.160000e+08,3.02,2.78,NaN,NaN,NaN,NaN,NaN,NaN,2016-08-05


In [103]:
kessan_table.columns

Index(['証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間', '期首', '期末',
       '名寄前勘定科目（売上高欄に掲載）', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益',
       '希薄化後一株当り純利益', '純資産又は株主資本', '総資産', '一株当り純資産', '営業キャッシュフロー',
       '投資キャッシュフロー', '財務キャッシュフロー', '情報公開又は更新日'],
      dtype='object')

In [142]:
kessan_table.dtypes

証券コード                        int64
企業名                         object
会計基準                        object
連結個別                        object
決算期                         object
決算期間                        object
期首                  datetime64[ns]
期末                  datetime64[ns]
名寄前勘定科目（売上高欄に掲載）            object
売上高                        float64
営業利益                       float64
経常利益                       float64
純利益                        float64
一株当り純利益                    float64
希薄化後一株当り純利益                float64
純資産又は株主資本                  float64
総資産                        float64
一株当り純資産                    float64
営業キャッシュフロー                 float64
投資キャッシュフロー                 float64
財務キャッシュフロー                 float64
情報公開又は更新日           datetime64[ns]
dtype: object

In [185]:
kessan_table.duplicated().any()

False

In [143]:
kessan_table.isnull().any()

証券コード               False
企業名                 False
会計基準                False
連結個別                False
決算期                 False
決算期間                False
期首                  False
期末                  False
名寄前勘定科目（売上高欄に掲載）     True
売上高                  True
営業利益                 True
経常利益                 True
純利益                  True
一株当り純利益              True
希薄化後一株当り純利益          True
純資産又は株主資本            True
総資産                  True
一株当り純資産              True
営業キャッシュフロー           True
投資キャッシュフロー           True
財務キャッシュフロー           True
情報公開又は更新日           False
dtype: bool

In [144]:
kessan_table.to_csv('/Users/Really/Stockyard/_csv/kessan_table.csv')

In [2]:
ahaha_table = pd.read_csv('/Users/Really/Stockyard/_dl_data/kabupro_ahaha_master_export.csv')

In [3]:
# まとめてできないの？
# kessan_table[['期首', '期末', '情報公開又は更新日']] = pd.to_datetime(kessan_table[['期首', '期末', '情報公開又は更新日']], format='%Y-%m-%d')
ahaha_table['期首'] = pd.to_datetime(ahaha_table['期首'], format='%Y-%m-%d')
ahaha_table['期末'] = pd.to_datetime(ahaha_table['期末'], format='%Y-%m-%d')
ahaha_table['情報公開又は更新日'] = pd.to_datetime(ahaha_table['情報公開又は更新日'], format='%Y-%m-%d')

In [4]:
ahaha_table = ahaha_table.sort_values(['証券コード', '期末']).reset_index()

In [7]:
ahaha_table = ahaha_table.drop(['index', 'Unnamed: 0'], axis=1)

In [11]:
ahaha_table.duplicated().any()

False

In [10]:
ahaha_table = ahaha_table.drop_duplicates().reset_index(drop=True)

In [12]:
ahaha_table

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日,有効項目数
0,1301,極洋,日本基準,連結,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.183500e+11,...,23.17,21.36,2.306900e+10,8.893700e+10,215.65,-2.340000e+09,-7.620000e+08,3.698000e+09,2016-05-09,22
1,1301,極洋,日本基準,連結,2016年3月期,第1四半期,2015-04-01,2015-06-30,売上高,5.016800e+10,...,3.02,2.78,NaN,NaN,NaN,NaN,NaN,NaN,2016-08-05,16
2,1301,極洋,日本基準,連結,2016年3月期,第2四半期,2015-04-01,2015-09-30,売上高,1.084990e+11,...,86.74,79.97,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-04,16
3,1301,極洋,日本基準,連結,2016年3月期,第3四半期,2015-04-01,2015-12-31,売上高,1.788900e+11,...,107.94,99.51,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-10,16
4,1301,極洋,日本基準,連結,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.266260e+11,...,171.33,157.94,2.306500e+10,9.460800e+10,2149.67,2.689000e+09,-5.114000e+09,2.482000e+09,2017-05-11,22
5,1301,極洋,日本基準,連結,2017年3月期,第1四半期,2016-04-01,2016-06-30,売上高,5.220600e+10,...,52.47,48.37,NaN,NaN,NaN,NaN,NaN,NaN,2017-08-04,16
6,1301,極洋,日本基準,連結,2017年3月期,第2四半期,2016-04-01,2016-09-30,売上高,1.095700e+11,...,112.99,104.16,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-06,16
7,1301,極洋,日本基準,連結,2017年3月期,第2四半期,2016-04-01,2016-09-30,売上高,1.095700e+11,...,112.99,104.16,2.360000e+10,1.065540e+11,NaN,NaN,NaN,NaN,2016-11-04,18
8,1301,極洋,日本基準,連結,2017年3月期,第3四半期,2016-04-01,2016-12-31,売上高,1.799750e+11,...,233.26,215.13,2.577900e+10,1.171680e+11,NaN,NaN,NaN,NaN,2017-02-10,18
9,1301,極洋,日本基準,連結,2017年3月期,通期,2016-04-01,2017-03-31,売上高,2.365610e+11,...,230.66,213.01,2.539100e+10,9.739100e+10,2378.09,6.010000e+08,-1.998000e+09,1.050000e+08,2017-05-11,22


In [162]:
ahaha_table.columns

Index(['index', 'Unnamed: 0', '証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間',
       '期首', '期末', '名寄前勘定科目（売上高欄に掲載）', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益',
       '希薄化後一株当り純利益', '純資産又は株主資本', '総資産', '一株当り純資産', '営業キャッシュフロー',
       '投資キャッシュフロー', '財務キャッシュフロー', '情報公開又は更新日', '有効項目数'],
      dtype='object')

In [161]:
ahaha_table.dtypes

index                        int64
Unnamed: 0                   int64
証券コード                        int64
企業名                         object
会計基準                        object
連結個別                        object
決算期                         object
決算期間                        object
期首                  datetime64[ns]
期末                  datetime64[ns]
名寄前勘定科目（売上高欄に掲載）            object
売上高                        float64
営業利益                       float64
経常利益                       float64
純利益                        float64
一株当り純利益                    float64
希薄化後一株当り純利益                float64
純資産又は株主資本                  float64
総資産                        float64
一株当り純資産                    float64
営業キャッシュフロー                 float64
投資キャッシュフロー                 float64
財務キャッシュフロー                 float64
情報公開又は更新日           datetime64[ns]
有効項目数                        int64
dtype: object

In [188]:
ahaha_table.to_csv('/Users/Really/Stockyard/_csv/kessan_table_ahaha.csv')